In [49]:
from geotiff import GeoTiff
import numpy as np
import csv
import shapefile
from PIL import Image, ImageDraw
import cv2

In [2]:
geotiff = GeoTiff("sentinel2img.tif")

In [3]:
geotiff.tif_shape

[4128, 3412, 4]

In [4]:
geotiff.get_coords(0, 0), geotiff.get_coords(4128, 3412)

((91.765286808, 26.151350309), (91.81163987682062, 26.113037162678296))

In [5]:
lon_array, lat_array = geotiff.get_coord_arrays()

In [6]:
lat_array = lat_array[:, 0]
lon_array = lon_array[0]

In [7]:
lat_array.shape

(4128,)

In [8]:
np.searchsorted(lon_array, 91.8038029981736)

3412

In [9]:
np.searchsorted(lon_array, 91.8037911877971)

3412

In [10]:
lon_array, lat_array

(array([91.76528681, 91.76529804, 91.76530927, ..., 91.80356627,
        91.8035775 , 91.80358873]),
 array([26.15135031, 26.15133908, 26.15132785, ..., 26.10503093,
        26.1050197 , 26.10500847]))

In [79]:
masks_array = np.zeros(lat_array.shape + lon_array.shape, dtype=float)
masks = Image.fromarray(masks_array)
masks_draw = ImageDraw.Draw(masks)

In [80]:
masks_array.shape, geotiff.get_coord_arrays()[0].shape

((4128, 3412), (4128, 3412))

In [81]:
with open("building_footprints_small.csv", "r", newline="") as csvfile:
    reader = csv.reader(csvfile, delimiter=",")
    next(reader)
    for row in reader:
        lat = float(row[0])
        lon = float(row[1])

        if lat < lat_array[0] and lon < lon_array[-1]:
            poly_cartesian = [tuple(map(float, i.split())) for i in row[4].strip("POLYGON((").strip(")").split("), (")[0].split(", ")]
            lon_indices = [np.searchsorted(lon_array, i[0]) for i in poly_cartesian]
            lat_indices = [np.searchsorted(lat_array, i[1], sorter=np.arange(len(lat_array) - 1, -1, -1)) for i in poly_cartesian]
            poly = list(zip(lon_indices, lat_indices))
            masks_draw.polygon(poly, fill=255)

In [82]:
unique, counts = np.unique(masks, return_counts=True)

In [83]:
dict(zip(unique, counts))

{0.0: 9893621, 255.0: 4191115}

In [77]:
masks = np.ndarray.astype(np.array(masks), np.uint8)
cv2.imwrite("buildings.png", masks)

True

In [84]:
masks = masks.convert("RGB")
masks.save("building.png")